In [ ]:
# 5k at each  100 m we can change
import numpy as np
total_dist = 5000
interval =100
shape = total_dist/interval
# ucs in mpa 50-250 lets model 50-100 at each 50


UCS =[]
RQD_per = []
RPM =[]
thrust =[]
noise=[]
ROP =[]

k_thrust =
k_RPM =
k =

for i in range(int(0, shape, 1)):
    sequence_num = i
    UCS[i] = np.random.rand(50, 250)
    RQD_per[i] = np.random.randint(20,100)
    thrust[i] = k_thrust * UCS[i]
    RPM[i] = k_RPM * (1/UCS[i])
    noise[i] = np.random.normal(0, stdev=9)
    ROP[i] = ((k*thrust*RPM)/UCS^1.5)  + noise[i]   # m/hr


# we need abrasion index??




df = {
    'SN':sequence_num,
    'UCS': UCS,

}

